# Preparing the IMDb dataset.

In [ ]:
import pandas as pd
import requests
import time

api_key = 'YOUR_KEY'

RANDOM_STATE = 1212

In [ ]:
# This block can be run once to create the necessary csv files

""" not_found = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])
imdb_data = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])

not_found.to_csv("not_found_final.csv", index=False)
imdb_data.to_csv("imdb_data_final.csv", index=False) """

In [ ]:
# This block makes API calls and save the data to csv files.
# Code works but is written very poorly and can be optimized.
# Remember to enter API-keys in the api_key variables.

not_found = pd.read_csv("not_found_final.csv")
imdb_data = pd.read_csv("imdb_data_final.csv")
cmu_data = pd.read_csv("cmu_backup.csv")

for index, row in cmu_data.iterrows():
    if len(imdb_data[imdb_data["genre"] == row["genre"]]) >= 1590:
        continue

    if row["movie_name"] in not_found["movie_name"].values and row["year"] in not_found["year"].values and row["genre"] in not_found["genre"].values:
        continue
    if row["movie_name"] in imdb_data["movie_name"].values and row["year"] in imdb_data["year"].values and row["genre"] in imdb_data["genre"].values:
        continue

    url = f"http://www.omdbapi.com/?t={row['movie_name']}&y={row['year']}&plot=full&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    print(data["Response"] + " " + row["movie_name"] + " " + str(row["year"]))

    time.sleep(0.5)

    if data["Response"] == "False":
        if data["Error"] == "Request limit reached!":
            print(data["Error"])
            break
        if data["Error"] == "Movie not found!":
            url = f"http://www.omdbapi.com/?t={row['movie_name']}&y={int(row['year']) - 1}&plot=full&apikey={api_key}"            
            response = requests.get(url)
            data = response.json()
            time.sleep(0.5)
            if data["Response"] == "True":
                if data["Plot"].count(".") >= 2:
                    imdb_data.loc[len(imdb_data)] = [row['movie_name'], int(row['year']), row['genre'], data['Plot']]
                else:
                    not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], data['Plot']]
            else:
                not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], row['plot']]
        else: 
            not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], row['plot']]
    else:
        if data["Plot"].count(".") >= 2:
            imdb_data.loc[len(imdb_data)] = [row['movie_name'], int(row['year']), row['genre'], data['Plot']]
        else:
            not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], data['Plot']]

# Saving the data to csv files
not_found.to_csv("not_found_final_Vx.csv", index=False)
imdb_data.to_csv("imdb_data_final_Vx.csv", index=False)